In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from numpy.core.fromnumeric import argsort, mean, squeeze
from torch import tensor
from torch.functional import norm
import matplotlib.pyplot as plt
import numpy as np
from torch import optim
import torch.nn.functional as F
import math as m
import time
import os 
import random
from PIL import Image
from torch.autograd import Variable
from PIL import Image
import numpy
import tensorflow as tf
from pathlib import Path
import pickle
import numpy as np
import torch
import torchvision
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive') 

if torch.cuda.is_available():
  device = torch.device("cuda:0")
  print("GPU")
else:
  device = torch.device("cpu")
  print("CPU")




class NLR3(nn.Module):
    def __init__(self,netin,netout,nethidden):
      super().__init__()
      self.netmodel= torch.nn.Sequential(torch.nn.Linear(netin, nethidden),torch.nn.Tanh(),torch.nn.Linear(nethidden, netout))
    def myforward (self,inv):
      outv=self.netmodel(inv)
      return outv


with open('/content/drive/My Drive/Master/Files/FeaturesToFiles172/Features172QueryStructureallF.txt', 'rb') as fp:
       AllData=pickle.load( fp)



target=[d['Target152F'] for d in AllData]
inp=[d['TargetCaptionF'] for d in AllData]
target=torch.tensor(target).to(device)
inp=torch.tensor(inp).to(device)

hidden=2500
l_r=0.2
epoch=100000
batch_size=900
save_duration=150
seed=100
min_error=0.1

model_mlp=NLR3(inp.shape[1],target.shape[1],hidden).to(device)
if(os.path.isfile('/content/drive/My Drive/Master/Files/SavedModels/UltraNetB152_2500.pth')):
  print('Load Saved Model From Google Drive')
  model_mlp.load_state_dict(torch.load( '/content/drive/My Drive/Master/Files/SavedModels/UltraNetB152_2500.pth', map_location=torch.device('cpu') ))

loss_fn2=torch.nn.CosineSimilarity()
loss_fn=torch.nn.MSELoss() 
optimizer=torch.optim.SGD(model_mlp.parameters(), lr=l_r)
s=0
sweep_range=inp.shape[0]%batch_size


optimizer.zero_grad()

if(os.path.isfile('/content/drive/My Drive/Master/Files/SavedModels/UltraNetBlosses1522.pkl')):
  with open('/content/drive/My Drive/Master/Files/SavedModels/UltraNetBlosses1522.pkl', 'rb') as fp:
       totallosses=pickle.load( fp)
       #print(totallosses)
else:
  totallosses=[]

for j in range(epoch):
  total_loss=0
  
  for l in range(int(inp.shape[0]/batch_size)):
    item_batch = inp[l*batch_size+s:(l+1)*batch_size+s,:].to(device)
    target_batch=target[l*batch_size+s:(l+1)*batch_size+s,:].to(device)
    netoutbatch=model_mlp.myforward(item_batch).to(device)
    
    loss = torch.mean(torch.abs(loss_fn2(target_batch,netoutbatch)))
    loss=1-loss
    loss.backward()
    optimizer.step()
    total_loss+=loss
  
  if (total_loss<min_error):
    break
  
  if(j%save_duration==0):
    print('iteration:',j, 'total loss',total_loss,'avg loss', total_loss/(inp.shape[0]/batch_size))
    torch.save(model_mlp.state_dict(), '/content/drive/My Drive/Master/Files/SavedModels/UltraNetB152_2500.pth')
    totallosses.append(total_loss)
    with open('/content/drive/My Drive/Master/Files/SavedModels/UltraNetBlosses1522.pkl', 'wb') as fp:
      pickle.dump( totallosses, fp)
    total_loss=0
    #print('model & Loss  saved ')
  
  
  
  s+=1
  if s==sweep_range:
      s=0
    

print('Finished Training')
torch.save(model_mlp.state_dict(), '/content/drive/My Drive/Master/Files/SavedModels/UltraFinal_NetB152_2500.pth')

Mounted at /content/drive
CPU


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


Load Saved Model From Google Drive
iteration: 0 total loss tensor(17.6385, grad_fn=<AddBackward0>) avg loss tensor(0.0923, grad_fn=<DivBackward0>)
iteration: 150 total loss tensor(17.6383, grad_fn=<AddBackward0>) avg loss tensor(0.0923, grad_fn=<DivBackward0>)
iteration: 300 total loss tensor(17.6382, grad_fn=<AddBackward0>) avg loss tensor(0.0923, grad_fn=<DivBackward0>)
iteration: 450 total loss tensor(17.6380, grad_fn=<AddBackward0>) avg loss tensor(0.0923, grad_fn=<DivBackward0>)
iteration: 600 total loss tensor(17.6378, grad_fn=<AddBackward0>) avg loss tensor(0.0923, grad_fn=<DivBackward0>)
iteration: 750 total loss tensor(17.6379, grad_fn=<AddBackward0>) avg loss tensor(0.0923, grad_fn=<DivBackward0>)
iteration: 900 total loss tensor(17.6378, grad_fn=<AddBackward0>) avg loss tensor(0.0923, grad_fn=<DivBackward0>)
iteration: 1050 total loss tensor(17.6377, grad_fn=<AddBackward0>) avg loss tensor(0.0923, grad_fn=<DivBackward0>)
iteration: 1200 total loss tensor(17.6377, grad_fn=<Ad